<a href="https://colab.research.google.com/github/joaohl19/premierleague_2018-19/blob/main/premierleague_2018_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preparo de dados da edição 2018/19 da Premier League para análise**


O preparo tem como intuito enriquecer o dataset Partidas com dados importantes para serem usados no treinamento de modelos preditivos.

Esses dados úteis podem variar de acordo com o que o modelo está tentando prever(gols, ganhador da partida, cartões...)

EX : Saber a média de gols de um time no campeonato é bastante útil no momento de prever quantos gols ele irá marcar em determinado jogo.

Como o dataset de partidas não possui tantas informações relativas àquele momento em que a partida iria ser jogada, é preciso calcular esses dados a partir de outros dados fornecidos

Os datasets usados foram disponibilizados pelo site https://footystats.org e podem ser baixados diretamente pelos links abaixo

Dataset Partidas : https://footystats.org/c-dl.php?type=matches&comp=1625

Dataset Times 1 : https://footystats.org/c-dl.php?type=teams&comp=1625

Dataset Times 2 : https://footystats.org/c-dl.php?type=teams2&comp=1625

## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
import copy

## Extração e limpeza dos dados

### Dataset Partidas


In [ ]:
partidas = pd.read_csv('/content/england-premier-league-matches-2018-to-2019-stats.csv')
partidas

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1533927600,Aug 10 2018 - 7:00pm,complete,74439,Manchester United,Leicester City,Andre Marriner,1,0.00,0.00,...,1.37,4.98,9.81,1.33,2.00,3.60,7.50,2.05,1.69,Old Trafford (Manchester)
1,1533987000,Aug 11 2018 - 11:30am,complete,51749,Newcastle United,Tottenham Hotspur,Martin Atkinson,1,0.00,0.00,...,4.51,3.77,1.84,1.31,1.95,3.50,5.75,1.83,1.87,St. James' Park (Newcastle upon Tyne)
2,1533996000,Aug 11 2018 - 2:00pm,complete,10353,AFC Bournemouth,Cardiff City,Kevin Friend,1,0.00,0.00,...,2.03,3.51,3.96,1.31,1.95,3.45,6.70,1.83,1.83,Vitality Stadium (Bournemouth- Dorset)
3,1533996000,Aug 11 2018 - 2:00pm,complete,24821,Fulham,Crystal Palace,Mike Dean,1,0.00,0.00,...,2.31,3.44,3.26,1.28,1.87,3.20,7.00,1.71,2.00,Craven Cottage (London)
4,1533996000,Aug 11 2018 - 2:00pm,complete,24121,Huddersfield Town,Chelsea,Chris Kavanagh,1,0.00,0.00,...,7.47,4.27,1.51,1.29,1.91,3.30,6.95,2.00,1.71,John Smith's Stadium (Huddersfield- West Yorks...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1557669600,May 12 2019 - 2:00pm,complete,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,...,1.32,5.80,9.00,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool)
376,1557669600,May 12 2019 - 2:00pm,complete,74457,Manchester United,Cardiff City,Jonathan Moss,38,2.00,0.61,...,1.28,6.20,10.00,1.12,1.41,2.00,3.60,1.74,2.05,Old Trafford (Manchester)
377,1557669600,May 12 2019 - 2:00pm,complete,30367,Southampton,Huddersfield Town,Lee Probert,38,1.22,0.33,...,1.41,4.90,7.50,1.19,1.61,2.50,4.75,1.80,2.00,St. Mary's Stadium (Southampton- Hampshire)
378,1557669600,May 12 2019 - 2:00pm,complete,60124,Tottenham Hotspur,Everton,Andre Marriner,38,2.06,1.06,...,2.10,3.50,3.55,1.22,1.71,2.80,6.00,1.59,2.30,Tottenham Hotspur Stadium (London)


In [ ]:
colunas = partidas.columns
colunas

Index(['timestamp', 'date_GMT', 'status', 'attendance', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Home)',
       'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_goal_timings', 'away_team_goal_timings',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'away_team_second_half_cards',
       'home_team_shots', 'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_te

In [ ]:
# Os únicos valores NaN ocorreram nas colunas que indicam os momentos em que a
# equipe marcou gol
for coluna in colunas:
  NaN = partidas[coluna].isna().sum()
  if(NaN > 0):
    print(coluna , NaN, sep = ": ")

home_team_goal_timings: 88
away_team_goal_timings: 119


In [ ]:
partidas.fillna(value = "No goal", inplace=True)

### Dataset Times 1


In [ ]:
times1 = pd.read_csv('/content/england-premier-league-teams-2018-to-2019-stats.csv')
times1

,team_name,common_name,season,country,matches_played,matches_played_home,matches_played_away,suspended_matches,wins,wins_home,...,goals_conceded_min_61_to_70,goals_conceded_min_71_to_80,goals_conceded_min_81_to_90,draw_percentage_overall,draw_percentage_home,draw_percentage_away,loss_percentage_ovearll,loss_percentage_home,loss_percentage_away,over145_corners_percentage
0,Arsenal FC,Arsenal,2018/2019,England,38,19,19,0,21,14,...,12,1,8,18,16,21,26,11,42,5
1,Tottenham Hotspur FC,Tottenham Hotspur,2018/2019,England,38,19,19,0,23,12,...,4,8,10,5,11,0,34,26,42,11
2,Manchester City FC,Manchester City,2018/2019,England,38,19,19,0,32,18,...,4,2,3,5,0,11,11,5,16,5
3,Leicester City FC,Leicester City,2018/2019,England,38,19,19,0,15,8,...,7,1,9,18,16,21,42,42,42,5
4,Crystal Palace FC,Crystal Palace,2018/2019,England,38,19,19,0,14,5,...,8,8,11,18,26,11,45,47,42,11
5,Everton FC,Everton,2018/2019,England,38,19,19,0,15,10,...,10,5,4,24,21,26,37,26,47,11
6,Burnley FC,Burnley,2018/2019,England,38,19,19,0,11,7,...,10,4,15,18,11,26,53,53,53,0
7,Southampton FC,Southampton,2018/2019,England,38,19,19,0,9,5,...,5,4,14,32,42,21,45,32,58,8
8,AFC Bournemouth,AFC Bournemouth,2018/2019,England,38,19,19,0,13,8,...,8,6,11,16,26,5,50,32,68,11
9,Manchester United FC,Manchester United,2018/2019,England,38,19,19,0,19,10,...,7,7,7,24,32,16,26,16,37,8


In [ ]:
times1.columns[0 : 100]

Index(['team_name', 'common_name', 'season', 'country', 'matches_played',
       'matches_played_home', 'matches_played_away', 'suspended_matches',
       'wins', 'wins_home', 'wins_away', 'draws', 'draws_home', 'draws_away',
       'losses', 'losses_home', 'losses_away', 'points_per_game',
       'points_per_game_home', 'points_per_game_away', 'league_position',
       'league_position_home', 'league_position_away', 'performance_rank',
       'goals_scored', 'goals_conceded', 'goal_difference', 'total_goal_count',
       'total_goal_count_home', 'total_goal_count_away', 'goals_scored_home',
       'goals_scored_away', 'goals_conceded_home', 'goals_conceded_away',
       'goal_difference_home', 'goal_difference_away',
       'minutes_per_goal_scored', 'minutes_per_goal_scored_home',
       'minutes_per_goal_scored_away', 'minutes_per_goal_conceded',
       'minutes_per_goal_conceded_home', 'minutes_per_goal_conceded_away',
       'clean_sheets', 'clean_sheets_home', 'clean_sheets_away'

In [ ]:
times1.columns[100 : 200]

Index(['goals_scored_per_match_home', 'goals_scored_per_match_away',
       'goals_conceded_per_match', 'goals_conceded_per_match_home',
       'goals_conceded_per_match_away', 'total_goals_per_match_half_time',
       'total_goals_per_match_half_time_home',
       'total_goals_per_match_half_time_away',
       'goals_scored_per_match_half_time',
       'goals_scored_per_match_half_time_home',
       'goals_scored_per_match_half_time_away',
       'goals_conceded_per_match_half_time',
       'goals_conceded_per_match_half_time_home',
       'goals_conceded_per_match_half_time_away', 'over05_count',
       'over15_count', 'over25_count', 'over35_count', 'over45_count',
       'over55_count', 'over05_count_home', 'over15_count_home',
       'over25_count_home', 'over35_count_home', 'over45_count_home',
       'over55_count_home', 'over05_count_away', 'over15_count_away',
       'over25_count_away', 'over35_count_away', 'over45_count_away',
       'over55_count_away', 'under05_count', 'un

In [ ]:
times1.columns[200: 293]

Index(['over25_half_time_percentage_home', 'over05_half_time_percentage_away',
       'over15_half_time_percentage_away', 'over25_half_time_percentage_away',
       'win_percentage', 'win_percentage_home', 'win_percentage_away',
       'home_advantage_percentage', 'clean_sheet_percentage',
       'clean_sheet_percentage_home', 'clean_sheet_percentage_away',
       'btts_percentage', 'btts_percentage_home', 'btts_percentage_away',
       'fts_percentage', 'fts_percentage_home', 'fts_percentage_away',
       'first_team_to_score_percentage', 'first_team_to_score_percentage_home',
       'first_team_to_score_percentage_away', 'clean_sheet_half_time',
       'clean_sheet_half_time_home', 'clean_sheet_half_time_away',
       'clean_sheet_half_time_percentage',
       'clean_sheet_half_time_percentage_home',
       'clean_sheet_half_time_percentage_away', 'fts_half_time',
       'fts_half_time_home', 'fts_half_time_away', 'fts_half_time_percentage',
       'fts_half_time_percentage_home', 'f

In [ ]:
# Não há nenhum NaN neste dataset
for coluna in times1.columns:
  NaN = times1[coluna].isna().sum()
  if(NaN > 0):
    print(coluna , NaN, sep = ": ")

### Dataset Times 2

In [ ]:
times2 = pd.read_csv('/content/england-premier-league-teams2-2018-to-2019-stats.csv')
times2

,team_name,common_name,season,country,average_attendance_overall,average_attendance_home,average_attendance_away,minutes_per_goal_scored_overall,minutes_per_goal_scored_home,minutes_per_goal_scored_away,...,btts_2h_away,btts_2h_percentage_overall,btts_2h_percentage_home,btts_2h_percentage_away,clean_sheets_2h_overall,clean_sheets_2h_home,clean_sheets_2h_away,failed_to_score_2h_overall,failed_to_score_2h_home,failed_to_score_2h_away
0,Arsenal FC,Arsenal,2018/2019,England,49412,59899,38925,47,41,55,...,6,28,26,32,18,12,6,14,4,10
1,Tottenham Hotspur FC,Tottenham Hotspur,2018/2019,England,45931,54216,37645,51,50,52,...,6,32,32,32,20,11,9,14,5,9
2,Manchester City FC,Manchester City,2018/2019,England,45914,54130,37698,36,30,45,...,3,21,26,16,27,14,13,9,2,7
3,Leicester City FC,Leicester City,2018/2019,England,34948,31851,38046,67,71,63,...,8,28,16,42,22,13,9,13,9,4
4,Crystal Palace FC,Crystal Palace,2018/2019,England,32193,25455,38932,67,90,54,...,10,39,26,53,14,8,6,15,10,5
5,Everton FC,Everton,2018/2019,England,38883,39043,38722,63,57,71,...,6,21,11,32,19,12,7,20,10,10
6,Burnley FC,Burnley,2018/2019,England,29307,20534,38080,76,71,81,...,6,28,26,32,16,8,8,22,11,11
7,Southampton FC,Southampton,2018/2019,England,33710,30083,37338,76,63,95,...,6,34,37,32,15,8,7,19,9,10
8,AFC Bournemouth,AFC Bournemouth,2018/2019,England,24718,10532,38904,61,57,66,...,2,16,21,11,14,8,6,22,10,12
9,Manchester United FC,Manchester United,2018/2019,England,56241,74498,37983,53,52,54,...,4,34,47,21,14,5,9,18,8,10


In [ ]:
times2.columns[0: 100]

Index(['team_name', 'common_name', 'season', 'country',
       'average_attendance_overall', 'average_attendance_home',
       'average_attendance_away', 'minutes_per_goal_scored_overall',
       'minutes_per_goal_scored_home', 'minutes_per_goal_scored_away',
       'minutes_per_goal_conceded_overall', 'minutes_per_goal_conceded_home',
       'minutes_per_goal_conceded_away', 'highest_goals_scored_home',
       'highest_goals_scored_away', 'highest_goals_conceded_home',
       'highest_goals_conceded_away', 'half_time_points_overall',
       'half_time_points_home', 'half_time_points_away',
       'corners_recorded_matches_num_overall',
       'corners_recorded_matches_num_home', 'cornersRecorded_matches_away',
       'corners_total_per_match_overall', 'corners_total_per_match_home',
       'corners_total_per_match_away', 'corners_against_overall',
       'corners_against_home', 'corners_against_away',
       'corners_against_per_match_overall', 'corners_against_per_match_home',
      

In [ ]:
times2.columns[100:200]

Index(['over45_corners_for_percentage_home',
       'over55_corners_for_percentage_home',
       'over65_corners_for_percentage_home',
       'over75_corners_for_percentage_home',
       'over85_corners_for_percentage_home', 'over25_corners_for_away',
       'over35_corners_for_away', 'over45_corners_for_away',
       'over55_corners_for_away', 'over65_corners_for_away',
       'over75_corners_for_away', 'over85_corners_for_away',
       'over25_corners_for_percentage_away',
       'over35_corners_for_percentage_away',
       'over45_corners_for_percentage_away',
       'over55_corners_for_percentage_away',
       'over65_corners_for_percentage_away',
       'over75_corners_for_percentage_away',
       'over85_corners_for_percentage_away', 'over25_corners_against_overall',
       'over35_corners_against_overall', 'over45_corners_against_overall',
       'over55_corners_against_overall', 'over65_corners_against_overall',
       'over75_corners_against_overall', 'over85_corners_against_o

In [ ]:
times2.columns[200: 300]

Index(['over05_cards_total_home', 'over15_cards_total_home',
       'over25_cards_total_home', 'over35_cards_total_home',
       'over45_cards_total_home', 'over55_cards_total_home',
       'over65_cards_total_home', 'over75_cards_total_home',
       'over85_cards_total_home', 'over05_cards_total_percentage_home',
       'over15_cards_total_percentage_home',
       'over25_cards_total_percentage_home',
       'over35_cards_total_percentage_home',
       'over45_cards_total_percentage_home',
       'over55_cards_total_percentage_home',
       'over65_cards_total_percentage_home',
       'over75_cards_total_percentage_home',
       'over85_cards_total_percentage_home', 'over05_cards_total_away',
       'over15_cards_total_away', 'over25_cards_total_away',
       'over35_cards_total_away', 'over45_cards_total_away',
       'over55_cards_total_away', 'over65_cards_total_away',
       'over75_cards_total_away', 'over85_cards_total_away',
       'over05_cards_total_percentage_away',
       '

In [ ]:
times2.columns[300: 400]

Index(['over15_cards_against_percentage_home',
       'over25_cards_against_percentage_home',
       'over35_cards_against_percentage_home',
       'over45_cards_against_percentage_home',
       'over55_cards_against_percentage_home',
       'over65_cards_against_percentage_home', 'over05_cards_against_away',
       'over15_cards_against_away', 'over25_cards_against_away',
       'over35_cards_against_away', 'over45_cards_against_away',
       'over55_cards_against_away', 'over65_cards_against_away',
       'over05_cards_against_percentage_away',
       'over15_cards_against_percentage_away',
       'over25_cards_against_percentage_away',
       'over35_cards_against_percentage_away',
       'over45_cards_against_percentage_away',
       'over55_cards_against_percentage_away',
       'over65_cards_against_percentage_away',
       'firstGoalScoredPercentage_overall', 'firstGoalScoredPercentage_home',
       'firstGoalScoredPercentage_away', 'shots_per_match_overall',
       'shots_per_m

In [ ]:
# Não há nenhum NaN neste dataset
for coluna in times2.columns:
  NaN = times2[coluna].isna().sum()
  if(NaN > 0):
    print(coluna , NaN, sep = ": ")

## Transformação


Escolhi as coluna mais relevantes entre as colunas dos datasets Times 1 e Times 2, de forma arbitrária, apenas para testar, imaginando que os dados seriam usados para prever se uma partida teria 2 ou mais gols



In [ ]:
# Selecionando as colunas relevantes do dataset Partidas
df = partidas.loc[:, ['timestamp', 'home_team_name', 'away_team_name', 'referee', 'Home Team Pre-Match xG',
                              'Away Team Pre-Match xG', 'over_15_percentage_pre_match', 'odds_ft_over15', 'average_goals_per_match_pre_match' ]]
df['goals'] = partidas['home_team_goal_count'] + partidas['away_team_goal_count']

### Coletando as métricas importantes inferidos através do dataset Partidas para juntá-los com as colunas selecionadas do dataset Partidas

In [ ]:
df_times = times1.loc[:, ['common_name','total_goal_count',
       'total_goal_count_home', 'total_goal_count_away', 'goals_scored_per_match', 'goals_scored_per_match_home', 'goals_scored_per_match_away',
       'goals_conceded_per_match', 'goals_conceded_per_match_home','goals_conceded_per_match_away','over15_count','over15_count_home', 'over15_count_away',
        'under15_count','under15_count_home', 'under15_count_away', 'clean_sheet_percentage',
       'clean_sheet_percentage_home', 'clean_sheet_percentage_away']]
df_times

,common_name,total_goal_count,total_goal_count_home,total_goal_count_away,goals_scored_per_match,goals_scored_per_match_home,goals_scored_per_match_away,goals_conceded_per_match,goals_conceded_per_match_home,goals_conceded_per_match_away,over15_count,over15_count_home,over15_count_away,under15_count,under15_count_home,under15_count_away,clean_sheet_percentage,clean_sheet_percentage_home,clean_sheet_percentage_away
0,Arsenal,124,58,66,1.92,2.21,1.63,1.34,0.84,1.84,34,18,16,4,1,3,21,37,5
1,Tottenham Hotspur,106,50,56,1.76,1.79,1.74,1.03,0.84,1.21,27,12,15,11,7,4,34,37,32
2,Manchester City,118,69,49,2.50,3.00,2.00,0.61,0.63,0.58,31,16,15,7,3,4,53,47,58
3,Leicester City,99,44,55,1.34,1.26,1.42,1.26,1.05,1.47,28,14,14,10,5,5,26,32,21
4,Crystal Palace,104,42,62,1.34,1.00,1.68,1.39,1.21,1.58,28,12,16,10,7,3,32,37,26
5,Everton,100,51,49,1.42,1.58,1.26,1.21,1.11,1.32,28,15,13,10,4,6,37,47,26
6,Burnley,113,56,57,1.18,1.26,1.11,1.79,1.68,1.89,31,17,14,7,2,5,21,26,16
7,Southampton,110,57,53,1.18,1.42,0.95,1.71,1.58,1.84,32,17,15,6,2,4,18,16,21
8,AFC Bournemouth,126,55,71,1.47,1.58,1.37,1.84,1.32,2.37,34,15,19,4,4,0,26,32,21
9,Manchester United,119,58,61,1.71,1.74,1.68,1.42,1.32,1.53,34,17,17,4,2,2,18,11,26


In [ ]:
df_times.columns

Index(['common_name', 'total_goal_count', 'total_goal_count_home',
       'total_goal_count_away', 'goals_scored_per_match',
       'goals_scored_per_match_home', 'goals_scored_per_match_away',
       'goals_conceded_per_match', 'goals_conceded_per_match_home',
       'goals_conceded_per_match_away', 'over15_count', 'over15_count_home',
       'over15_count_away', 'under15_count', 'under15_count_home',
       'under15_count_away', 'clean_sheet_percentage',
       'clean_sheet_percentage_home', 'clean_sheet_percentage_away'],
      dtype='object')

In [ ]:
"""
Dicionário para manter o registro das métricas importantes de cada time em
casa e fora
"""
dict_home = {}
dict_away = {}
for x in df_times['common_name']:
  dict_home[x] = [0,0, 0, 0]
  dict_away[x] = [0,0, 0, 0]
dict_home

{'Arsenal': [0, 0, 0, 0],
 'Tottenham Hotspur': [0, 0, 0, 0],
 'Manchester City': [0, 0, 0, 0],
 'Leicester City': [0, 0, 0, 0],
 'Crystal Palace': [0, 0, 0, 0],
 'Everton': [0, 0, 0, 0],
 'Burnley': [0, 0, 0, 0],
 'Southampton': [0, 0, 0, 0],
 'AFC Bournemouth': [0, 0, 0, 0],
 'Manchester United': [0, 0, 0, 0],
 'Liverpool': [0, 0, 0, 0],
 'Chelsea': [0, 0, 0, 0],
 'West Ham United': [0, 0, 0, 0],
 'Watford': [0, 0, 0, 0],
 'Newcastle United': [0, 0, 0, 0],
 'Cardiff City': [0, 0, 0, 0],
 'Fulham': [0, 0, 0, 0],
 'Brighton & Hove Albion': [0, 0, 0, 0],
 'Huddersfield Town': [0, 0, 0, 0],
 'Wolverhampton Wanderers': [0, 0, 0, 0]}

In [ ]:
"""
Dicionário para registrar o número de partidas jogadas por cada time no
momento em que aconteceu a partida
"""
games = {}
for x in df_times['common_name']:
  games[x] = 0
games

{'Arsenal': 0,
 'Tottenham Hotspur': 0,
 'Manchester City': 0,
 'Leicester City': 0,
 'Crystal Palace': 0,
 'Everton': 0,
 'Burnley': 0,
 'Southampton': 0,
 'AFC Bournemouth': 0,
 'Manchester United': 0,
 'Liverpool': 0,
 'Chelsea': 0,
 'West Ham United': 0,
 'Watford': 0,
 'Newcastle United': 0,
 'Cardiff City': 0,
 'Fulham': 0,
 'Brighton & Hove Albion': 0,
 'Huddersfield Town': 0,
 'Wolverhampton Wanderers': 0}

In [ ]:
df_times = df_times.drop(columns=['common_name'])

In [ ]:
# Inicializando o dicionário que vai armazenar os valores importantes
data = {}
for index2, row2 in partidas.iterrows():
  if(row2['home_team_name'] not in data):
     data[row2['home_team_name']] = {}
     # Iterando sobre o número de rodadas
     for i in range (39):
         data[row2['home_team_name']][str(i)] = {}
         # Iterando sobre o número de colunas
         for att in df_times.columns:
            data[row2['home_team_name']][str(i)][att] = 0

In [ ]:
"""
consultar o dataset PARTIDAS

 atributos gerais
    'total_goal_count' -> contar gols do time
    'goals_scored_per_match' -> contar gols do time e dividir pelo número de jogos
    'goals_conceded_per_match' -> contar gols do time e dividir pelo número de jogos
    'over15_count' -> somar gols do mandante e gols do visitante e ver se é maior que 1
    'under15_count' -> somar gols do mandante e gols do visitante e ver se é menor que 1
    'clean_sheet_percentage' -> contar o número de partidas sem tomar gol e dividir
                                pelo número de partidas jogadas

 atributos casa
    'total_goal_count_home'
    'goals_scored_per_match_home'
    'goals_conceded_per_match_home'
    'over15_count_home'
    'under15_count_home'
    'clean_sheet_percentage_home'

 atributos fora
    'total_goal_count_away'
    'goals_conceded_per_match_away'
    'goals_scored_per_match_away'
    'over15_count_away'
    'under15_count_away'
    'clean_sheet_percentage_away'

Como os datasets pelos quais eu iterei são relativamente pequenos,
usei o método .iterrows() nessa e em outras iterações, apesar de
ele ser um método custoso

"""

# Primeiro `for` para as rodadas
for index2, row2 in partidas.iterrows():

  home_team = row2['home_team_name']
  away_team = row2['away_team_name']

  dict_home[home_team][0] += row2['home_team_goal_count'] # gols feitos em casa pelo time da casa
  dict_home[home_team][1] += row2['away_team_goal_count'] # gols sofridos em casa pelo time da casa
  dict_home[home_team][2] += 1  # número de partidas jogadas em casa pelo time
  dict_home[home_team][3] += 1 if row2['away_team_goal_count'] == 0 else 0 # número de cleansheets em casa do time

  dict_away[away_team][0] += row2['away_team_goal_count'] # gols feitos fora de casa pelo time de fora
  dict_away[away_team][1] += row2['home_team_goal_count'] # gols sofridos fora de casa pelo time de fora
  dict_away[away_team][2] += 1  # número de partidas jogadas fora de casa pelo time
  dict_away[away_team][3] += 1 if row2['home_team_goal_count'] == 0 else 0  # número de cleansheets fora de casa do time

  games_awayTeam = dict_home[away_team][2] + dict_away[away_team][2]
  games_homeTeam = dict_home[home_team][2] + dict_away[home_team][2]


  # Segundo `for` para os atributos
  for att in df_times.columns:
      home_att = att.find('home')
      away_att = att.find('away')

      # Atributos do mandante
      if home_att != -1:
        data[away_team][str(games_awayTeam)][att] = data[away_team][str(games_awayTeam - 1)][att]
        data[home_team][str(games_homeTeam)][att] = data[home_team][str(games_homeTeam - 1)][att]
        if(att == 'total_goal_count_home'):
          data[home_team][str(games_homeTeam)][att] += row2['home_team_goal_count'] + row2['away_team_goal_count']

        elif(att == 'goals_scored_per_match_home'):
          data[home_team][str(games_homeTeam)][att] = (dict_home[home_team][0] / dict_home[home_team][2])

        elif(att == 'goals_conceded_per_match_home'):
          data[home_team][str(games_homeTeam)][att] = (dict_home[home_team][1] / dict_home[home_team][2])

        elif(att == 'under15_count_home'):
          data[home_team][str(games_homeTeam)][att] +=  (0 if (row2['home_team_goal_count'] + row2['away_team_goal_count']) > 1 else 1)

        elif(att == 'clean_sheet_percentage_home'):
          data[home_team][str(games_homeTeam)][att] =  (dict_home[home_team][3] * 100) / dict_home[home_team][2]

        else:
            data[home_team][str(games_homeTeam)][att] += (1 if (row2['home_team_goal_count'] + row2['away_team_goal_count']) > 1 else 0)

      # Atributos do visitante
      elif away_att != -1:
        data[away_team][str(games_awayTeam)][att] = data[away_team][str(games_awayTeam - 1)][att]
        data[home_team][str(games_homeTeam)][att] = data[home_team][str(games_homeTeam - 1)][att]
        if(att == 'total_goal_count_away'):
          data[away_team][str(games_awayTeam)][att] += row2['home_team_goal_count'] + row2['away_team_goal_count']

        elif(att == 'goals_scored_per_match_away'):
          data[away_team][str(games_awayTeam)][att] = (dict_away[away_team][0] / dict_away[away_team][2])

        elif(att == 'goals_conceded_per_match_away'):
          data[away_team][str(games_awayTeam)][att] = (dict_away[away_team][1] / dict_away[away_team][2])

        elif(att == 'under15_count_away'):
          data[away_team][str(games_awayTeam)][att] += (0 if (row2['home_team_goal_count'] + row2['away_team_goal_count']) > 1 else 1)

        elif(att == 'clean_sheet_percentage_away'):
          data[away_team][str(games_awayTeam)][att] = (dict_away[away_team][3] * 100) / dict_away[away_team][2]

        else:
            data[away_team][str(games_awayTeam)][att] += (1 if (row2['home_team_goal_count'] + row2['away_team_goal_count']) > 1 else 0)

      # Atributos gerais
      else:
        if(att == 'total_goal_count'):
          data[home_team][str(games_homeTeam)][att] = (row2['home_team_goal_count'] + row2['away_team_goal_count']) + data[home_team][str(games_homeTeam - 1)][att]
          data[away_team][str(games_awayTeam)][att] = (row2['home_team_goal_count'] + row2['away_team_goal_count']) + data[away_team][str(games_awayTeam - 1)][att]

        elif(att == 'goals_scored_per_match'):
          data[home_team][str(games_homeTeam)][att] = ((dict_home[home_team][0] + dict_away[home_team][0]) / games_homeTeam)
          data[away_team][str(games_awayTeam)][att] = ((dict_home[away_team][0] + dict_away[away_team][0]) / games_awayTeam)

        elif(att == 'goals_conceded_per_match'):
          data[home_team][str(games_homeTeam)][att] = ((dict_home[home_team][1] + dict_away[home_team][1]) / games_homeTeam)
          data[away_team][str(games_awayTeam)][att] = ((dict_home[away_team][1] + dict_away[away_team][1]) / games_awayTeam)

        elif(att == 'under15_count'):
          data[home_team][str(games_homeTeam)][att] = data[home_team][str(games_homeTeam - 1)][att] +  (0 if (row2['home_team_goal_count'] + row2['away_team_goal_count']) > 1 else 1)
          data[away_team][str(games_awayTeam)][att] = data[away_team][str(games_awayTeam - 1)][att] + (0 if (row2['home_team_goal_count'] + row2['away_team_goal_count']) > 1 else 1)

        elif(att == 'clean_sheet_percentage'):

          data[home_team][str(games_homeTeam)][att] = ((dict_home[home_team][3] + dict_away[home_team][3])  * 100) / (dict_home[home_team][2] + dict_away[home_team][2])
          data[away_team][str(games_awayTeam)][att] = ((dict_home[away_team][3] + dict_away[away_team][3])  * 100) / (dict_home[away_team][2] + dict_away[away_team][2])

        else:
          data[home_team][str(games_homeTeam)][att] = data[home_team][str(games_homeTeam - 1)][att] + (1 if (row2['home_team_goal_count'] + row2['away_team_goal_count']) > 1 else 0)
          data[away_team][str(games_awayTeam)][att] = data[away_team][str(games_awayTeam - 1)][att] + (1 if (row2['home_team_goal_count'] + row2['away_team_goal_count']) > 1 else 0)


In [ ]:
# Convertendo o dicionário para uma matriz
data_2d = []
for time, subdic in data.items():
    for jogo, subsubdic in subdic.items():
        for métrica, valor in subsubdic.items():
            data_2d.append([time, jogo, métrica, valor])

# Convertendo a matriz de dados para um DataFrame
df_data = pd.DataFrame(data_2d, columns=['Time', 'Número do jogo', 'Métrica', 'Valor'])
df_data

,Time,Número do jogo,Métrica,Valor
0,Manchester United,0,total_goal_count,0.000000
1,Manchester United,0,total_goal_count_home,0.000000
2,Manchester United,0,total_goal_count_away,0.000000
3,Manchester United,0,goals_scored_per_match,0.000000
4,Manchester United,0,goals_scored_per_match_home,0.000000
...,...,...,...,...
14035,Crystal Palace,38,under15_count_home,7.000000
14036,Crystal Palace,38,under15_count_away,3.000000
14037,Crystal Palace,38,clean_sheet_percentage,31.578947
14038,Crystal Palace,38,clean_sheet_percentage_home,36.842105


### Junção dos dados inferidos com os dados do dataset Partidas em um único DataFrame

In [ ]:
# Juntando os dados do dataset partidas com os dados coletados
for index, row in df.iterrows():
  home_team = row['home_team_name']
  away_team = row['away_team_name']
  df2_home = df_data[(df_data['Número do jogo'] == str(games[home_team])) & (df_data['Time'] == home_team)]
  df2_away = df_data[(df_data['Número do jogo'] == str(games[away_team])) & (df_data['Time'] == away_team)]

  for att in df2_home['Métrica']:
    away_att = att.find('away')
    if(away_att == -1):
        df2_home_aux = df2_home[df2_home['Métrica'] == att].Valor
        value = np.array(df2_home_aux)[0]
        df.loc[index : index, (att + '_homeTeam')] = value

  for att in df2_away['Métrica']:
    home_att = att.find('home')
    if(home_att == -1):
        df2_away_aux = df2_away[df2_away['Métrica'] == att].Valor
        value = np.array(df2_away_aux)[0]
        df.loc[index : index, (att + '_awayTeam')] = value

  games[home_team] += 1
  games[away_team] += 1

In [ ]:
df

,timestamp,home_team_name,away_team_name,referee,Home Team Pre-Match xG,Away Team Pre-Match xG,over_15_percentage_pre_match,odds_ft_over15,average_goals_per_match_pre_match,goals,...,goals_scored_per_match_awayTeam,goals_scored_per_match_away_awayTeam,goals_conceded_per_match_awayTeam,goals_conceded_per_match_away_awayTeam,over15_count_awayTeam,over15_count_away_awayTeam,under15_count_awayTeam,under15_count_away_awayTeam,clean_sheet_percentage_awayTeam,clean_sheet_percentage_away_awayTeam
0,1533927600,Manchester United,Leicester City,Andre Marriner,0.00,0.00,0,1.33,0.00,3,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
1,1533987000,Newcastle United,Tottenham Hotspur,Martin Atkinson,0.00,0.00,0,1.31,0.00,3,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
2,1533996000,AFC Bournemouth,Cardiff City,Kevin Friend,0.00,0.00,0,1.31,0.00,2,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
3,1533996000,Fulham,Crystal Palace,Mike Dean,0.00,0.00,0,1.28,0.00,2,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
4,1533996000,Huddersfield Town,Chelsea,Chris Kavanagh,0.00,0.00,0,1.29,0.00,3,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1557669600,Liverpool,Wolverhampton Wanderers,Martin Atkinson,1.96,1.27,81,1.18,2.92,2,...,1.270270,1.055556,1.189189,1.277778,30.0,14.0,7.0,4.0,24.324324,11.111111
376,1557669600,Manchester United,Cardiff City,Jonathan Moss,1.67,1.10,78,1.12,2.72,2,...,0.864865,0.611111,1.864865,1.722222,27.0,12.0,10.0,6.0,24.324324,22.222222
377,1557669600,Southampton,Huddersfield Town,Lee Probert,1.46,1.11,84,1.19,3.06,2,...,0.567568,0.611111,2.027027,2.444444,26.0,14.0,11.0,4.0,13.513514,11.111111
378,1557669600,Tottenham Hotspur,Everton,Andre Marriner,1.71,1.32,64,1.22,2.53,4,...,1.405405,1.222222,1.189189,1.277778,27.0,12.0,10.0,6.0,37.837838,27.777778


## Carregamento em um arquivo .csv

In [ ]:
csv_path = "/content/teams_stats_for_goals.csv"
df.to_csv(csv_path)